In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

Using TensorFlow backend.


In [ ]:
model = load_model(r'C:\Users\gokul\Desktop\New folder\emojinator.h5')

def Emojinator():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[-2]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

def get_emojis():
    emojis_folder = r'C:\Users\gokul\Desktop\New folder/hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
Emojinator()

0
1
2
3
4
5
6
7
8
9
10
11
12
5 0.7157297
5 0.97304463
5 0.9995517
5 0.999974
5 0.9929992
5 0.99927324
5 0.9878072
5 0.955317
5 0.9662905
5 0.9691808
5 0.9685373
5 0.99970955
5 0.9983494
5 0.99974996
5 0.9990965
5 0.9284847
9 0.5903981
5 0.928418
5 0.99985814
5 0.99510163
5 0.99510163
5 0.98442334
5 0.9980413
5 0.99480176
5 0.9230371
5 0.9994723
5 0.9975789
5 0.93708944
5 0.99881965
5 0.93657637
5 0.93657637
5 0.9960459
5 0.977478
5 0.9870964
5 0.91304666
5 0.7449558
5 0.9921028
5 0.5523127
5 0.96408606
5 0.9995819
5 0.9995819
5 0.99807405
5 0.9980971
5 0.9994431
5 0.9965581
5 0.99287367
5 0.9985588
7 0.6208604
5 0.9993926
5 0.9691122
5 0.9691122
5 0.9996549
5 0.99946815
5 0.912049
5 0.94653535
5 0.5996972
7 0.6645402
5 0.99845886
5 0.96104544
5 0.9617771
5 0.92279106
5 0.92279106
5 0.8083224
5 0.9565179
5 0.9995272
7 0.57886094
5 0.99968076
5 0.86521196
5 0.9950454
5 0.9999417
7 0.98987985
7 0.98987985
5 0.7485143
5 0.9015258
5 0.99998367
5 0.9994647
5 0.60240513
5 0.9966647
5 0.937622

5 0.99889034
5 0.99035186
5 0.9998977
7 0.79401076
5 0.5992611
5 0.999995
7 0.9486816
5 0.92643094
5 0.96145475
5 0.96145475
5 0.9470828
5 0.9174312
5 0.9919219
6 0.9447491
5 0.9819535
5 0.9996113
7 0.928839
5 0.9983461
5 0.9983461
5 0.9913033
5 0.9999752
5 0.6866356
5 0.9584275
5 0.99993396
5 0.9989454
5 0.9925454
6 0.88610214
5 0.99957424
5 0.99957424
5 0.98228204
9 0.90175945
6 0.7104611
5 0.9998392
5 0.9339706
5 0.9939037
5 0.9966928
5 0.9989182
5 0.9999906
5 0.9999906
5 0.9987413
5 0.9998778
5 0.9652915
5 0.9982066
5 0.6419072
5 0.96614754
5 0.9939824
5 0.99995947
5 0.778944
5 0.9963282
5 0.9963282
5 0.9985917
5 0.88959634
5 0.9706523
5 0.9951047
5 0.96238095
5 0.9892475
5 0.9945293
5 0.667213
5 0.99410003
5 0.99410003
5 0.5000515
5 0.99515164
5 0.99721897
7 0.9265991
5 0.99281764
5 0.92899096
5 0.99733233
5 0.99170625
5 0.99404055
7 0.566405
5 0.99142843
5 0.99984646
5 0.98976994
5 0.99821836
5 0.9998411
5 0.99999523
7 0.9996997
7 0.87535363
5 0.9960139
5 0.9989606
5 0.7452267
5 

5 0.9833793
6 0.9949421
9 0.97265744
9 0.99999845
9 0.9724708
10 0.7523386
10 0.9357816
6 0.65567315
6 0.5118197
7 0.60204095
6 0.99995196
10 0.5462197
7 0.49726588
6 0.9730311
6 0.43119183
5 0.74738234
5 0.6980618
5 0.64913285
6 0.8383076
4 0.79091084
6 0.9991794
6 0.6611253
6 0.6785829
6 0.4413726
6 0.99881965
6 0.69225866
6 0.7240897
4 0.6545145
9 0.67249423
6 0.994131
6 0.6638767
6 0.43321484
6 0.43321484
5 0.9923919
7 0.5494833
9 0.99473524
9 0.99304646
9 0.9820862
9 0.9999279
9 0.99946517
9 0.99946517
9 0.9999913
9 0.9999913
6 0.5562368
9 0.9995434
10 0.46167907
9 0.9189097
9 0.9219133
6 0.9581295
9 0.99950576
9 0.99950576
9 0.6457497
5 0.9765355
6 0.9968027
6 0.9968027
9 0.99974114
9 0.9999764
6 0.99975044
9 0.999998
5 0.9983767
9 0.9999633
9 0.9999633
5 0.99641454
5 0.9815845
5 0.9684569
5 0.7740199
5 0.99935836
5 0.9988005
5 0.99997246
5 0.99985576
5 0.99656206
5 0.99656206
5 0.79955155
5 0.99967706
5 0.88218373
5 0.9993685
5 0.9998672
5 0.97987545
5 0.87341815
7 0.98683935
6 

1 0.6264425
1 0.6264425
1 0.89119756
1 0.9499777
1 0.98584414
1 0.98051983
1 0.97518504
1 0.97518504
1 0.98644626
1 0.9809307
1 0.99586606
1 0.9960609
1 0.9960609
1 0.9960794
1 0.99391407
1 0.9883384
1 0.9883384
1 0.97634333
1 0.97243416
1 0.95899093
1 0.9571416
1 0.9507613
1 0.9507613
1 0.9533673
1 0.96240634
1 0.970212
1 0.9862278
1 0.99204683
1 0.99282795
1 0.99036586
1 0.9999267
1 0.9999267
1 0.9926823
1 0.9998443
1 0.9998093
1 0.99989915
1 0.99989915
1 0.9998667
1 0.9998498
1 0.9998528
1 0.9998652
1 0.99983406
1 0.99983406
1 0.99982774
1 0.9992551
1 0.99750584
1 0.99750584
1 0.9960305
1 0.9927214
1 0.97430927
1 0.9559888
1 0.9559888
1 0.7887767
1 0.5745826
1 0.742033
11 0.7437902
11 0.7437902
11 0.4544208
11 0.46714956
11 0.69797105
11 0.71425015
11 0.71425015
11 0.6818779
11 0.5145871
11 0.58052284
11 0.57190377
11 0.8053036
11 0.8053036
11 0.55941564
11 0.5104774
11 0.78847635
11 0.78847635
11 0.6020279
11 0.6395446
11 0.6639532
11 0.7528476
11 0.7707876
11 0.7707876
11 0.798164

1 0.9999393
1 0.9999393
1 0.9999368
1 0.99995005
1 0.9998827
1 0.9998827
1 0.9998994
1 0.9998171
1 0.9998103
1 0.9998292
1 0.99987674
1 0.99987674
1 0.99994123
1 0.9999497
1 0.99963236
1 0.99989486
1 0.99989486
1 0.99992156
1 0.9998944
1 0.9996705
1 0.9995578
1 0.9233897
1 0.9502198
1 0.7292145
1 0.75169957
1 0.75169957
1 0.9146413
1 0.86995375
1 0.939238
1 0.939238
1 0.9807091
1 0.98379046
1 0.99986434
1 0.9999547
1 0.99996006
1 0.9995807
1 0.9977107
1 0.9886027
1 0.9699948
1 0.9699948
1 0.9314253
11 0.4153291
11 0.3775369
11 0.3775369
8 0.72255814
8 0.72255814
8 0.7290271
8 0.6124461
8 0.5362104
8 0.52980095
8 0.6081339
8 0.55501455
8 0.6403687
8 0.64777905
8 0.669368
8 0.62711114
8 0.67633176
8 0.67633176
8 0.43575913
1 0.90807796
1 0.91416705
1 0.9387868
1 0.9387868
1 0.965349
1 0.91874343
1 0.9164878
1 0.8367466
1 0.93753976
1 0.91916525
1 0.92294884
1 0.9637953
1 0.9562722
1 0.9688403
1 0.9057296
1 0.9229471
1 0.9229471
8 0.45217624
8 0.63881916
8 0.6999636
8 0.6585016
8 0.695086

...

3 0.94112504
3 0.95358586
3 0.9731844
3 0.906805
3 0.91436374
3 0.9017862
3 0.97190577
3 0.97070104
3 0.97566926
3 0.9819864
3 0.87196124
3 0.9804166
3 0.96584576
3 0.9014594
3 0.9535754
3 0.9806311
3 0.79270697
3 0.9661193
3 0.96655834
3 0.9191872
3 0.9835175
3 0.9789964
3 0.9490038
3 0.94034773
3 0.81932014
3 0.958165
3 0.9810626
3 0.693473
3 0.9592948
3 0.5239472
3 0.8601118
3 0.8601118
3 0.9651025
3 0.84790665
3 0.9214902
3 0.98398626
3 0.9829438
3 0.94148916
3 0.91505295
3 0.9672665
3 0.95231795
3 0.9831874
3 0.9831874
3 0.96543694
3 0.93894565
3 0.86933213
3 0.9393874
3 0.9869136
3 0.9611624
3 0.93339413
3 0.9886536
3 0.9615986
3 0.9615986
3 0.85610896
3 0.9656063
3 0.964668
3 0.9771484
3 0.9319947
3 0.95593274
3 0.97904783
3 0.8480714
3 0.90828985
3 0.90828985
3 0.9056623
3 0.95691746
3 0.9711906
3 0.96658397
3 0.9514833
3 0.95415956
3 0.9580815
3 0.96211296
3 0.94893736
3 0.94893736
3 0.9793985
3 0.91090554
3 0.6270295
3 0.9886048
3 0.96913785
3 0.9787344
3 0.9743249
3 0.797237

1 0.99890685
1 0.99890685
1 0.99988294
1 0.99977523
1 0.9995173
1 0.99907315
1 0.99907315
1 0.99926084
1 0.99975413
1 0.99971455
1 0.9995413
1 0.9995413
1 0.99787605
1 0.9925801
1 0.997042
1 0.99628484
1 0.9982503
1 0.9982503
1 0.9992092
1 0.99438405
1 0.99689734
1 0.99814355
1 0.99814355
1 0.9953531
1 0.9996068
1 0.9976144
1 0.9976144
1 0.993469
1 0.9975076
1 0.99723154
1 0.99925786
1 0.99925786
1 0.9977843
1 0.99633956
1 0.9999223
1 0.99978405
1 0.99978405
1 0.999567
1 0.99984765
1 0.99922585
1 0.999406
1 0.99906045
1 0.99906045
1 0.99787235
1 0.9983364
1 0.9998622
1 0.9998216
1 0.9998216
1 0.9998481
1 0.9995932
1 0.9993136
1 0.9993136
1 0.9994165
1 0.9994165
1 0.9997696
1 0.9992071
1 0.9993462
1 0.9993462
1 0.99987245
1 0.9998826
1 0.99985814
1 0.99990404
1 0.99990404
1 0.9996331
1 0.9993482
1 0.9983052
1 0.99765646
1 0.9986155
1 0.9986155
1 0.9998802
1 0.9991652
1 0.9998109
1 0.9998109
1 0.99911827
1 0.99885774
1 0.9998542
1 0.9998542
1 0.99804795
1 0.9998745
1 0.9987847
1 0.996736

3 0.9905874
4 0.7412673
4 0.7412673
11 0.5845179
9 0.50721866
11 0.324987
11 0.324987
7 0.69128156
7 0.87376565
4 0.48185423
11 0.28085193
11 0.85057765
11 0.85057765
11 0.9022918
3 0.41226503
10 0.5119296
4 0.2429912
4 0.2429912
9 0.383333
4 0.890595
4 0.8247939
4 0.8247939
4 0.77538395
4 0.77538395
4 0.8324365
4 0.9769214
4 0.9993104
4 0.9993104
4 0.99976426
4 0.99980015
4 0.91735536
4 0.91735536
7 0.7093427
7 0.94967884
7 0.85169363
7 0.86650735
7 0.9635385
7 0.9635385
7 0.9481372
7 0.8754868
7 0.59611714
7 0.9013581
7 0.9013581
5 0.50856465
5 0.99725693
5 0.9962519
5 0.9962519
3 0.90887636
3 0.90887636
9 0.96755046
9 0.72800255
9 0.7398534
9 0.7398534
9 0.72533375
3 0.6641828
3 0.9606559
3 0.9702862
3 0.9702862
3 0.9810384
3 0.98113513
3 0.9674516
3 0.92446774
3 0.6106505
3 0.6106505
4 0.75232655
4 0.85950553
4 0.95569664
4 0.95569664
4 0.98250705
4 0.98250705
4 0.9989222
4 0.99833566
4 0.9992723
4 0.9992723
4 0.99934167
4 0.99802554
4 0.99903893
4 0.99903893
4 0.99783474
4 0.99734

7 0.971612
7 0.9752108
7 0.94206405
7 0.94206405
7 0.77326787
7 0.77326787
7 0.8234711
7 0.89642006
7 0.84156865
7 0.84156865
7 0.7525787
7 0.6565814
7 0.7127137
7 0.8285078
7 0.8285078
7 0.72955453
7 0.7585961
7 0.5648863
4 0.5190627
7 0.75529915
7 0.75529915
7 0.7977219
7 0.6914742
7 0.5491859
7 0.5491859
7 0.5038887
7 0.6164368
7 0.5481808
7 0.5092637
7 0.623566
7 0.623566
4 0.64750195
4 0.66724735
4 0.7220735
4 0.7220735
4 0.75511557
4 0.52451944
4 0.7603229
4 0.637356
4 0.637356
7 0.6321761
7 0.6321761
7 0.5276888
4 0.5554578
4 0.6387664
4 0.6387664
4 0.7283869
4 0.9228462
4 0.99273235
4 0.99273235
4 0.9570114
4 0.9570114
6 0.49119705
9 0.8851089
4 0.9826857
4 0.9826857
4 0.99130356
4 0.9262046
4 0.66746646
3 0.88813084
3 0.88813084
3 0.9587086
3 0.9275317
3 0.83883536
3 0.89053106
3 0.83410096
3 0.83410096
3 0.91850734
3 0.86256087
3 0.92350644
3 0.92350644
3 0.8588497
3 0.77528644
3 0.80035496
3 0.7978064
3 0.83154154
3 0.83154154
3 0.7759558
3 0.8328149
3 0.5757822
3 0.5757822


5 0.8313311
7 0.7996738
11 0.99491113
